# Run Resnet50 Inference benchmarks on OpneVINO with synthetic data

### Write benchmark commands into a file benchmark_openvino-rn50.sh

In [ ]:
%cd

In [ ]:
%%writefile benchmark_openvino-rn50.sh
lscpu
# activate opensource tf environment
source ~/opentfov/bin/activate
source  /opt/intel/openvino/bin/setupvars.sh
# get trained model from intel repo
cd ~/oneapi_workshop
wget https://download.01.org/opencv/public_models/012020/resnet-50-tf/resnet_v1-50.pb
#convert the tensorflow model to openVINO format
python3 /opt/intel/openvino/deployment_tools/model_optimizer/mo_tf.py --input_shape=[1,224,224,3] --mean_values=[123.68,116.78,103.94] --input=map/TensorArrayStack/TensorArrayGatherV3 --output=softmax_tensor --input_model=$HOME/oneapi_workshop/resnet_v1-50.pb --reverse_input_channels
# run openVINO benchmarks
# run for batch size 1 and 1 stream and notedown FPS
$HOME/oneapi_workshop/samples/intel64/Release/benchmark_app -m $HOME/oneapi_workshop/resnet_v1-50.xml -nstreams 1 -niter 100 -b 1
# run for batch size 1 and 4 streams and notedown FPS
$HOME/oneapi_workshop/samples/intel64/Release/benchmark_app -m $HOME/oneapi_workshop/resnet_v1-50.xml -nstreams 4 -niter 100 -b 1
# run for batch size 16 and 1 stream and notedown FPS
$HOME/oneapi_workshop/samples/intel64/Release/benchmark_app -m $HOME/oneapi_workshop/resnet_v1-50.xml -nstreams 1 -niter 100 -b 16
# run for batch size 16 and 4 streams and notedown FPS
$HOME/oneapi_workshop/samples/intel64/Release/benchmark_app -m $HOME/oneapi_workshop/resnet_v1-50.xml -nstreams 4 -niter 100 -b 16
deactivate

### Check content of benchmark_inteltf.sh

In [ ]:
!cat benchmark_openvino-rn50.sh

### Optional step : Remove all old output files

In [ ]:
!rm -rf benchmark_openvino-rn50.sh.*

### Submit to queue.

In [ ]:
!qsub benchmark_openvino-rn50.sh -l nodes=1:ppn=2 -d .

### check job status

In [ ]:
!qstat

### If the job status is finished, check the the output file with proper output name.

In [ ]:
!cat benchmark_openvino-rn50.sh.o*

### Notedown the number of images/sec for training and inference

In [ ]:
!grep -e "Xeon" -e "batch size" -e "inference requests" -e "Throughput"  benchmark_openvino-rn50.sh.o*

### close the notebook